### Steps to record a Run

1. Run CARLA
2. RUN carla_node.py
3. Once carla_node is ready, come here and run the cell 1 and 2 below.

The saved frames should occupy 1 mb per second of simulated run. So shouldnt take up much space. 

In [1]:
# CELL 1

import random
import sys
import time
import carla
from carla import ColorConverter as cc
import numpy as np
import numpy
from PIL import Image

client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
# grp = get_global_planner(world=world, planner_resolution=2.0)
Map = world.get_map()

# camera sensor callback function, gets and image and saves it to the folder
def to_bgra_array(image):
    """Convert a CARLA raw image to a BGRA numpy array."""
    array = numpy.frombuffer(image.raw_data, dtype=numpy.dtype("uint8"))
    array = numpy.reshape(array, (image.height, image.width, 4))
    return array

def to_rgb_array(image):
    """Convert a CARLA raw image to a RGB numpy array."""
    array = to_bgra_array(image)
    # Convert BGRA to RGB.
    array = array[:, :, :3]
    array = array[:, :, ::-1]
    return array

def process_img(image):
    '''Process sensor data and save as jpg'''
    PATH = OUTPUT_PATH + ('%08d' % image.frame_number) +'.jpg'
    im = to_rgb_array(image)
    im = Image.fromarray(im)
    im.save(PATH, quality=QUALITY)
    
def get_camera(ego_vehicle):
    camera_bp =  world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(WIDTH))
    camera_bp.set_attribute('image_size_y', str(HEIGHT))

    transform = carla.Transform(carla.Location(x=-15.5, z=12.5), carla.Rotation(pitch=8.0))
    Attachment = carla.AttachmentType

    camera = world.spawn_actor(camera_bp, transform, attach_to=ego_vehicle, attachment_type=Attachment.SpringArm)
    camera.listen(process_img)
    
    return camera

def get_ego_vehicle():
    world.wait_for_tick(seconds=10)
    ego_vehicle = None
    
    while ego_vehicle == None:
        all_vehicles = world.get_actors().filter('vehicle.*')
        for vehicle in all_vehicles:
            if vehicle.attributes['role_name'] == EGO_NAME:
                ego_vehicle = vehicle
                break
                
        if ego_vehicle is not None: break
            
    return ego_vehicle

4. Change the parameter of the run_name below and execute the cells, the frames will get recorded in the folder when the simulation steps

In [5]:
# CELL 2

RUN_NAME = 'LEFT_LANE_CHANGE_1'  # DEF CHANGE EACH RUN
OUTPUT_PATH = '/home/grasp/Desktop/videos/' + RUN_NAME + '/' # change save path

# should be mostly fine, talk to rohan before changing, 
HEIGHT = 720 #720p
WIDTH = 1280
QUALITY = 45
EGO_NAME = 'ego'

5. Now run fp_node or point_to_point_nav_node run the cell below. The timeout on the cell below is 60s, so make sure the simulation starts moving things within 60s. 

In [4]:
# waits for 60s for the sim to start running 
world.wait_for_tick(seconds=60) 

while True: #loops over multiple runs
    ego_vehicle = get_ego_vehicle()
    camera = get_camera(ego_vehicle)
    
    try:
        while True:
            world.wait_for_tick(seconds=60)
            
            # ego id may change between runs, so need to reattach camera
            if ego_vehicle.get_location().x == 0.000000:
                camera.destroy()
                ego_vehicle = get_ego_vehicle()
                print(ego_vehicle)
                camera = get_camera(ego_vehicle)

    except KeyboardInterrupt:
        camera.destroy()
        print('Exit')
    finally:
        camera.destroy()
        break

Actor(id=1562, type=vehicle.tesla.model3)


### After running the cell above

6. Run fp_node.py or point_to_point based on scenario (if model load takes more then 60 seconds. the above cell will timeout, in which case rerun the cell)
7. When you want to stop recording, stop the cell (restart the kernel).

## CONVERT RECORDING TO VIDEO

1. In terminal, navigate to the folder where you saved the current run
2. Run this command:
   ffmpeg -r 45 -pattern_type glob -i '*.jpg' -vb 2M -vcodec libx265 video.mp4 
   
   Arguments (mostly recommended, but feel free to experiment):
   -r: frames per second (since our sim works at 20fps, 45 fps in video is 2.25x fast)
   -pattern_type: just finds all .jpg images in a folder
   -vb: video bitrate, basically video quality/compression control, 
   -vcodec: x265 is the latest, smallest file size
   
3. Rename the file
   -filename is video.mp4, rename it to the following format
   
   SCENARIO_TRAFFICDENSITY_DIFFICULTY_RUNNUMBER.MP4
       SCENARIO: "RIGHTTURN", "LEFTTURN", "LANECHANGERIGHT", "GOSTRAIGHT", "P2PROUTE8", "P2PROUTE2", etc
       TRAFFICDENSITY: "HIGH" for 200, "MED" for 150, "LOW" for 100
       DIFFICULTY: Describe the scenario out of the following tags: EASY, HARD, MEDIUM. hard scenariou would include some good negotiation for that scenario. 
   
   example: RIGHTTURN_HIGH_HARD_01.mp4, P2PROUTE5_LOW_EASY_03.mp4
   
4. Upload the file: upload the videos to the SVD_VIDEOS folder in SHARED DRIVE. 